In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [ ]:
##Load the dataset
data=pd.read_csv("Churn_Modelling.csv")
data.head()

In [ ]:
#preprocessing of data
#drop irrelevent column
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)


In [ ]:
data.head()

In [ ]:
#Encoding the categorical column 
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data

In [ ]:
#One  hOt Encoder of country column
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder()
geoEncoder=onehot_encoder_geo.fit_transform(data[['Geography']])
geoEncoder

In [ ]:
geoEncoder.toarray()

In [ ]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

In [ ]:
geo_encoded_df=pd.DataFrame(geoEncoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

In [ ]:
#combine new geography column and removing the orignal one
data=pd.concat([data.drop(['Geography'],axis=1),geo_encoded_df],axis=1)
data.head()

In [ ]:
#save the encoders and scalar 
with open('label_encoder_gender.pkl','wb')as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

In [ ]:
#divide the dataset into indepandent features
X=data.drop('Exited',axis=1)
y=data['Exited']

#split it into test_train sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Scale these features
scalar=StandardScaler()
X_train=scalar.fit_transform(X_train)
X_test=scalar.transform(X_test)


In [ ]:
X_train

In [ ]:
X_test

In [ ]:
with open('scalar.pkl','wb') as file:
    pickle.dump(scalar,file) 

In [ ]:
data

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [ ]:
#building model ANN
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),  #1st hidden layerconnected to input layer
    Dense(32,activation='relu'),  #2nd hidden layer
    Dense(1,activation='sigmoid')  #output layer
]
)


In [ ]:
model.summary( ) 
#parameter are combination of bias and weights

In [ ]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()
loss

In [ ]:
#compile the model
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

In [ ]:
#setup the tensorBoard for training
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
log_dir="logs/fit/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [ ]:
#Set up Eary Stoppings
early_stopping_callbacks=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [ ]:
#Training the model
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callbacks]
)

In [ ]:
model.save('model.h5')

In [ ]:
#Load TensorBoard Extension
%load_ext tensorboard



In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
#LOAD the pickle file
